# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,upernavik,72.7868,-56.1549,-27.61,90,81,3.04,GL,1679894426
1,1,kaitangata,-46.2817,169.8464,17.47,56,84,5.82,NZ,1679894466
2,2,yellowknife,62.4560,-114.3525,-23.71,70,0,1.54,CA,1679894375
3,3,puerto asis,0.5133,-76.5007,20.59,96,73,0.20,CO,1679894467
4,4,vaini,-21.2000,-175.2000,27.09,94,75,2.57,TO,1679894467


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    hover_cols=["City", "Humidity"]
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df['Max Temp'] > 21) & 
                                    (city_data_df['Max Temp'] < 27) &
                                    (city_data_df['Wind Speed'] < 4.5) &
                                    (city_data_df['Cloudiness'] > 0) &
                                    (city_data_df['Cloudiness'] < 25)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
69,69,busselton,-33.6500,115.3333,23.52,36,1,0.77,AU,1679894570
155,155,jaque,7.5175,-78.1703,25.38,85,14,1.23,PA,1679894685
178,178,cabedelo,-6.9811,-34.8339,24.37,78,24,3.94,BR,1679894765
182,182,garachine,8.0647,-78.3628,25.14,90,19,3.56,PA,1679894767
202,202,maragogi,-9.0122,-35.2225,24.89,84,21,2.37,BR,1679894775


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
69,busselton,AU,-33.6500,115.3333,36,
155,jaque,PA,7.5175,-78.1703,85,
178,cabedelo,BR,-6.9811,-34.8339,78,
182,garachine,PA,8.0647,-78.3628,90,
202,maragogi,BR,-9.0122,-35.2225,84,
312,lazaro cardenas,MX,17.9583,-102.2000,81,
349,puerto escondido,MX,15.8500,-97.0667,83,
357,wanning,CN,18.8003,110.3967,77,
398,taoudenni,ML,22.6783,-3.9836,18,
399,khudumelapye,BW,-23.8833,24.7500,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

counter = 1

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    response = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Wait for 5 seconds after every 3rd record
    counter += 1
    if counter == 3:
        time.sleep(5)
        counter = 0
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
busselton - nearest hotel: Gale Street Motel & Villas
jaque - nearest hotel: Ana Mar Lodge
cabedelo - nearest hotel: Pousada Belo Mar
garachine - nearest hotel: No hotel found
maragogi - nearest hotel: Pousada Mariluz
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
puerto escondido - nearest hotel: Hotel Barlovento
wanning - nearest hotel: No hotel found
taoudenni - nearest hotel: Taoudeni
khudumelapye - nearest hotel: No hotel found
araouane - nearest hotel: No hotel found
kahului - nearest hotel: Maui Seaside Hotel
malabo - nearest hotel: Cati Paludismo Bebi
carlos chagas - nearest hotel: No hotel found
marzuq - nearest hotel: No hotel found
luoyang - nearest hotel: 一见客栈


,City,Country,Lat,Lng,Humidity,Hotel Name
69,busselton,AU,-33.6500,115.3333,36,Gale Street Motel & Villas
155,jaque,PA,7.5175,-78.1703,85,Ana Mar Lodge
178,cabedelo,BR,-6.9811,-34.8339,78,Pousada Belo Mar
182,garachine,PA,8.0647,-78.3628,90,No hotel found
202,maragogi,BR,-9.0122,-35.2225,84,Pousada Mariluz
312,lazaro cardenas,MX,17.9583,-102.2000,81,Hotel Sol del Pacífico
349,puerto escondido,MX,15.8500,-97.0667,83,Hotel Barlovento
357,wanning,CN,18.8003,110.3967,77,No hotel found
398,taoudenni,ML,22.6783,-3.9836,18,Taoudeni
399,khudumelapye,BW,-23.8833,24.7500,49,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=0.8,
    color="City",
    hover_cols=["City", "Country", "Humidity", "Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)